In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('steam_games.csv')
df = df[df['name'].notna()]
df.drop(["discount_price", "publisher", "developer", "release_date", "desc_snippet", "types", "achievements", "recent_reviews", "mature_content"], axis=1, inplace=True)

In [3]:
for row in df.iterrows():
    if pd.isna(row[1][3]) and pd.isna(row[1][6]):
        print("deleting: ", row[1][1])
        df.drop(row[0], inplace=True)

deleting:  Ghost In The Shell: Stand Alone Complex 2nd Gig
deleting:  Ghost In The Shell: Stand Alone Complex Season 1
deleting:  Gintama Season 4
deleting:  CRYENGINE - Sample Assets
deleting:  Worms Reloaded: The "Pre-order Forts and Hats" DLC Pack
deleting:  Krater - Soundtrack
deleting:  Dustforce Soundtrack
deleting:  The Incredible Adventures of Van Helsing II - OST
deleting:  D4: SWERY's Choice Costume Set -4 Cups of Coffee-
deleting:  D4: SWERY's Choice Costume Set -4 Bottles of Tequila-
deleting:  Kemono Friends Season 1
deleting:  New Game Season 1
deleting:  Showa Genroku Rakugo Shinju Season 1
deleting:  Con Man Season 1
deleting:  DayZ Tools
deleting:  Forge of Gods: Ultimate Titan Bundle
deleting:  Killing Floor "London's Finest" Character Pack
deleting:  Dungeon Defenders Capture the Flag Pre-Alpha Pass (Free DLC)
deleting:  Digital Sound Factory: Session Drummer 3 Expansions
deleting:  Mob Psycho 100 Season 1
deleting:  Re:ZERO -Starting Life in Another World- Season 1


In [4]:
duplicated_items = df[df["name"].duplicated()]["name"].values.tolist()
games = {}
print(df.shape)
for row in df.iterrows():
    if row[1][1] in duplicated_items:
        if row[1][1] in games:
            games[row[1][1]].append(row[1].isna().sum())
        else:
            games[row[1][1]] = [row[1].isna().sum()]
for item in games:
    games[item].sort()
    count = 0
    for row in df.iterrows():
        if row[1][1] == item and (row[1].isnull().sum() == games[item][0]):
            df.drop(row[0], inplace=True)
            count += 1
            print("deleting: ", row[1][1])
        
        if count + 1 == len(games[item]):
            break
print(df.shape)
df.to_csv("steam_games_dup_removed.csv", index=False)

(40675, 11)
deleting:  RUSH
deleting:  Project Highrise
deleting:  Batman: Arkham City - Game of the Year Edition
deleting:  Angels of Death
deleting:  Luna
deleting:  Flashback
deleting:  New York Bus Simulator
deleting:  Warhammer Quest
deleting:  City Builder
deleting:  Killing Time
deleting:  Jigsaw Puzzles
deleting:  Hide and Seek
deleting:  Hidden Object Adventure Bundle
deleting:  Arcade Bundle
deleting:  Arcade Bundle
deleting:  Soundtrack Edition
deleting:  Solitaire Collection
deleting:  Ultimate Arena
deleting:  Black Rose
deleting:  Solstice
deleting:  Surge
deleting:  Game & Soundtrack
deleting:  7 Days in Dream 光尘
deleting:  Bounce
deleting:  Game + Soundtrack
deleting:  Chaos Theory
deleting:  Deluxe Edition
deleting:  Deluxe Edition
deleting:  Rumpus
deleting:  Space Maze
deleting:  Spark
deleting:  The Mine
deleting:  Memories
deleting:  Ashes
deleting:  Dodge
deleting:  We Were Here Too: Supporter Edition
deleting:  The Deer
deleting:  Zombie Apocalypse
deleting:  Aft

In [5]:
df = pd.read_csv('steam_games_dup_removed.csv')

all_review_cols = ["all_review_sentiment", "all_review_sentiment_num", "all_review_num"]
temp_df = []
idx = 0
for row in df.iterrows():
    if row[1][2] == None or pd.isna(row[1][2]):
        temp_df.append(["None", 0, 0])
    else:
        try:
            items = row[1][2].split('(')
            sentiment = items[0].lower().strip()[0:-1]
            items = items[1].split(')')
            total = int(items[0].strip().replace(',', ''))
            percent = int(items[1].split('%')[0].strip()[3:]) / 100
            temp_df.append([sentiment, int(total * percent), total])
        except:
            temp_df.append(["None", 0, 0])
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
all_review_df = pd.DataFrame(temp_df, columns=all_review_cols)
df.drop(["all_reviews"], axis=1, inplace=True)
df = pd.concat([df, all_review_df], axis=1)

df['all_review_sentiment'] = df['all_review_sentiment'].astype('category').cat.codes
df['popular_tags'].fillna(df['genre'], inplace=True)

for i in range(len(df)):
    if pd.isna(df.loc[i, 'genre']):
        df.drop(i, inplace=True)
        
df.drop(['minimum_requirements', 'recommended_requirements', 'game_description'], axis=1, inplace=True)
df.to_csv("steam_games_reviews.csv", index=False)

categories = list(df['all_review_sentiment'].astype('category').cat.categories)

12.271790341578328 %
24.541126815861798 %
36.810463290145265 %
49.07979976442874 %
61.34913623871221 %
73.61847271299568 %
85.88780918727915 %
98.15714566156262 %


In [6]:
def format_csv(data):
    try:
        return list(set(data.split(',')))
    except:
        return []

df = pd.read_csv('steam_games_reviews.csv')
df['total'] = df['popular_tags'] + df['genre'] + df['game_details']
df['total'] = df['total'].apply(format_csv)
df['languages'] = df['languages'].apply(format_csv)
df.drop(['popular_tags', 'game_details', 'genre'], axis=1, inplace=True)

df.to_csv('steam_games_genres.csv', index=False)

In [7]:
df = pd.read_csv('steam_games_genres.csv')
df['all_review_sentiment'] =(df['all_review_sentiment']-df['all_review_sentiment'].min())/(df['all_review_sentiment'].max()-df['all_review_sentiment'].min())
df['all_review_sentiment_num'] =(df['all_review_sentiment_num']-df['all_review_sentiment_num'].min())/(df['all_review_sentiment_num'].max()-df['all_review_sentiment_num'].min())
df['all_review_num'] =(df['all_review_num']-df['all_review_num'].min())/(df['all_review_num'].max()-df['all_review_num'].min())

df.to_csv('steam_game_review_normal.csv', index=False)

In [8]:
df = pd.read_csv('steam_game_review_normal.csv')

def format_price(price):
    price_str = str(price)
    if price_str[0] == "$":
        return float(price_str[1:])
    
    try:
        test = int(price_str[0])
        return float(price_str)
    except:
        return 0

df['original_price'] = df['original_price'].apply(format_price)

df.to_csv('steam_games_final_format.csv', index=False)

In [9]:
df

,url,name,languages,original_price,all_review_sentiment,all_review_sentiment_num,all_review_num,total
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"['Russian', 'Japanese', 'Polish', 'Traditional...",19.99,1.000000,0.014711,0.013751,"['Fast-Paced', 'Steam Cloud', 'Gore', 'Sci-fi'..."
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"['Portuguese', 'Russian', 'Japanese', 'Polish'...",29.99,0.111111,0.154052,0.270377,"['Adventure', 'Multiplayer', 'Tactical', 'FPS'..."
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"['Russian', 'French', 'German', 'English']",39.99,0.333333,0.001876,0.002272,"['Adventure', 'Steam Cloud', 'RPG', 'Sci-fi', ..."
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"['Russian', 'Czech', 'English', 'Spanish - Spa...",44.99,0.111111,0.038308,0.054009,"['Adventure', 'Steam Cloud', 'Multiplayer', 'A..."
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"['French', 'Russian', 'German', 'English']",0.00,0.333333,0.003192,0.003710,"['RPG', 'Sci-fi', 'Multiplayer', 'PvE', 'Tacti..."
...,...,...,...,...,...,...,...,...
40323,https://store.steampowered.com/app/899836/Rock...,Rocksmith® 2014 Edition – Remastered – Sabaton...,"['Japanese', 'English', 'Spanish - Spain', 'It...",2.99,0.000000,0.000000,0.000000,"['Partial Controller Support', 'Steam Cloud', ..."
40324,https://store.steampowered.com/app/899832/Rock...,Rocksmith® 2014 Edition – Remastered – Stone T...,"['Japanese', 'English', 'Spanish - Spain', 'It...",2.99,0.000000,0.000000,0.000000,"['Partial Controller Support', 'Steam Cloud', ..."
40325,https://store.steampowered.com/app/906840/Fant...,Fantasy Grounds - Quests of Doom 4: A Midnight...,['English'],7.99,0.000000,0.000000,0.000000,"['Turn-Based', 'RPG', 'Strategy', 'Cross-Platf..."
40326,https://store.steampowered.com/app/906635/Mega...,Mega Man X5 Sound Collection,"['Japanese', 'Traditional Chinese', 'English',...",9.99,0.000000,0.000000,0.000000,"['Steam Leaderboards', 'Steam Cloud', 'Steam T..."
